# Datenbereinigung und -vorbereitung
1. [Strom-Messwerte vorbereiten](#1)
1. [Innentempteratur vorbereiten](#2)
1. [Wetterdaten vorbereiten](#3)
1. [Feiertage vorbereiten](#4)
1. [Umfragedaten vorbereiten](#5)
1. [Datensatz auf ausgewählte Häuser beschränken](#6)
1. [Daten zusammenführen](#7)

In [109]:
import numpy as np
import pandas as pd
import os
from datetime import date, timedelta

print("Setup complete")

Setup complete


<a id="1"></a>
## 1. Strom-Messwerte vorbereiten
Gesamtverbrauch berechnen: [Pecan Street Inc. Data Use / Dataport FAQ](https://docs.google.com/document/d/1_9H9N4cgKmJho7hK8nii6flIGKPycL7tlWEtd4UhVEQ/edit#heading=h.nhqpscy1c9sm)

In [110]:
main_data = pd.read_csv('../input/2023-12-20/data_austin_15min.csv')

# Umrechnung des Zeitstempels in UTC, da verschiedene Zeitzonen +
# Entfernen der Zeitzone und Umwandlung in naives datetime-Objekt
main_data['timestamp'] = pd.to_datetime(main_data.local_15min, utc=True).dt.tz_localize(None)

# Gesamtverbrauch des Hauses berechnen
main_data['use'] = main_data[['grid', 'solar']].sum(axis=1, skipna=True)

# Entfernen unnötiger Spalten
columns_to_drop = ['local_15min', 'grid', 'solar', 'leg1v', 'leg2v']
main_data.drop(columns=columns_to_drop, inplace=True)

main_data.sample(5)

,dataid,air1,air2,air3,airwindowunit1,aquarium1,bathroom1,bathroom2,bedroom1,bedroom2,...,sprinkler1,sumppump1,utilityroom1,venthood1,waterheater1,waterheater2,wellpump1,winecooler1,timestamp,use
657603,8156,0.000,NaN,NaN,NaN,NaN,0.005,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-03 19:00:00,0.497
819979,9922,1.216,-0.005,NaN,NaN,NaN,NaN,NaN,0.005,0.006,...,NaN,NaN,0.003,NaN,NaN,NaN,NaN,NaN,2018-06-24 21:00:00,1.845
113727,2361,-0.002,NaN,NaN,NaN,NaN,0.001,NaN,NaN,NaN,...,NaN,NaN,NaN,0.008,NaN,NaN,NaN,NaN,2018-04-11 00:45:00,0.572
682295,8386,-0.001,NaN,NaN,NaN,NaN,NaN,NaN,0.304,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-07-19 01:00:00,1.162
534152,7800,0.001,NaN,NaN,NaN,NaN,NaN,NaN,0.121,NaN,...,NaN,NaN,NaN,0.018,NaN,NaN,NaN,NaN,2018-04-24 16:15:00,0.940


<a id="2"></a>
## 2. Innentempteratur vorbereiten

In [111]:
# Daten aus mehreren Dateien zusammenführen
folder_path = '../input/2023-12-20/indoor_temp'
temp_dfs = list()

for file in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    temp_dfs.append(df)

indoor_temp_data = pd.concat(temp_dfs, ignore_index=True)

# Zeitstempel erstellen
indoor_temp_data['timestamp'] = pd.to_datetime(indoor_temp_data.localminute)

# Auswahl der relevanten Spalten
relevant_columns = ['dataid', 'timestamp', 'temp_c']
indoor_temp_data = indoor_temp_data[relevant_columns]

# Spalte umbenennen
indoor_temp_data.rename(columns={'temp_c': 'indoor_temp'}, inplace=True)

indoor_temp_data.sample(5)

,dataid,timestamp,indoor_temp
54947372,9643,2014-09-19 18:22:01,23.31
1792166,7893,2014-05-01 22:36:02,23.94
18374652,3504,2014-10-05 20:50:01,23.12
67702169,6012,2014-09-27 23:01:01,23.94
6654396,5456,2014-08-15 06:56:01,29.12


<a id="3"></a>
## 3. Wetterdaten vorbereiten

In [112]:
def convert_fahrenheit_to_celsius(fahrenheit):
    """
    Funktion zur Umrechnung von Grad Fahrenheit zu Grad Celsius.
    :param fahrenheit: Temperatur in Grad Fahrenheit.
    :return: Temperatur in Grad Celsius.
    """
    celsius = round((fahrenheit - 32) * 5 / 9, 2)
    return celsius

In [113]:
weather_data = pd.read_csv('../input/2023-12-20/weather.csv')

# Daten für Stadt Austin filtern
weather_data = weather_data[(weather_data.latitude == 30.292432) & (weather_data.longitude == -97.699662)]

# Duplikate filtern
weather_data = weather_data.groupby('localhour').last().reset_index()

# Umrechnung von Grad Fahrenheit zu Grad Celsius
weather_data['outdoor_temp'] = weather_data.temperature.apply(convert_fahrenheit_to_celsius)
weather_data['app_outdoor_temp'] = weather_data.apparent_temperature.apply(convert_fahrenheit_to_celsius)

# Zeitstempel erstellen
weather_data['timestamp'] = pd.to_datetime(weather_data.localhour)

# Auswahl der relevanten Spalten
relevant_columns = ['timestamp', 'outdoor_temp', 'app_outdoor_temp', 'humidity']
weather_data = weather_data[relevant_columns]

weather_data.sample(5)

,timestamp,outdoor_temp,app_outdoor_temp,humidity
76686,2019-10-07 02:00:00,25.74,26.20,0.70
1185,2011-02-19 09:00:00,16.56,16.56,0.90
18127,2013-01-25 11:00:00,20.98,20.98,0.70
317,2011-01-14 05:00:00,3.11,3.11,0.57
5967,2011-09-06 16:00:00,32.96,30.60,0.13


<a id="4"></a>
## 4. Feiertage vorbereiten
Feiertage in Austin: https://www.austintexas.gov/department/official-city-holidays

Die Feiertage umfassen:
* Neujahr (erster Wochentag im neuen Jahr)
* Martin-Luther-King-Tag (dritter Montag im Januar)
* Tag des Präsidenten (dritter Montag im Februar)
* Memorial Day (letzter Montag im Mai)
* Juneteenth (19. Juni)
* Unabhängigkeitstag (4. Juli)
* Tag der Arbeit (erster Montag im September)
* Veteranentag (11. November)
* Erntedankfest (vierter Donnerstag im November)
* Thanksgiving-Freitag (Tag nach Thanksgiving)
* Heiligabend (24. Dezember)
* Weihnachtstag (25. Dezember)

Anmerkung: Sollten Neujahr, der Unabhängigkeitstag oder Weihnachten auf einen Sonntag fallen, dann ist der Tag danach ebenfalls ein Feiertag. Wenn einer dieser Tage auf einen Samstag fällt, dann wird der Tag davor zum Feiertag (Quelle: https://usa.usembassy.de/feiertage.htm)

In [114]:
def find_specific_weekday_of_month(nth, weekday, month, year):
    """
    Function to find a specific occurrence of a weekday in a given month and year. 
    If nth is negative, it finds the last occurrence of the weekday in the month.

    :param nth: int, the occurrence of the weekday in the month. Negative for the last occurrence.
    :param weekday: int, the weekday (0=Monday, 1=Tuesday, ..., 6=Sunday).
    :param month: int, the month (1=January, 2=February, ..., 12=December).
    :param year: int, the year.
    :return: datetime.date, the date of the specific occurrence of the weekday in the specified month and year.
    """
    if nth < 0:
        # Start with the first day of the next month
        if month == 12:
            holiday_date = date(year + 1, 1, 1)
        else:
            holiday_date = date(year, month + 1, 1)

        # Subtract days until the desired weekday is found
        while holiday_date.weekday() != weekday:
            holiday_date -= timedelta(days=1)
    else:
        # Start with the first day of the month
        holiday_date = date(year, month, 1)

        # Find the first occurrence of the weekday in the month
        while holiday_date.weekday() != weekday:
            holiday_date += timedelta(days=1)

        # Find the nth occurrence
        nth_weekday = holiday_date + timedelta(days=(nth - 1) * 7)
        return nth_weekday

    return holiday_date

In [115]:
def get_holidays(year):
    """
    Erstellt eine Liste der Feiertage für ein gegebenes Jahr in Austin, Texas, USA.
    Die Funktion berücksichtigt spezielle Regeln für bewegliche Feiertage sowie Anpassungen für Feiertage,
    die auf das Wochenende fallen.

    :param year: Das Jahr, für das die Feiertage berechnet werden sollen. Es sollte ein ganzzahliges Jahr sein (z.B. 2022).
    :return: Eine Liste mit Datumsangaben aller Feiertage für ein gegebenes Jahr in Austin, Texas, USA.
    """
    holidays = {
        "Neujahrstag": date(year, 1, 1),
        "Martin-Luther-King-Tag": find_specific_weekday_of_month(3, 0, 1, year),
        "Tag des Präsidenten": find_specific_weekday_of_month(3, 0, 2, year),
        "Memorial Day": find_specific_weekday_of_month(-1, 0, 5, year),
        "Juneteenth": date(year, 6, 19),
        "Unabhängigkeitstag": date(year, 7, 4),
        "Tag der Arbeit": find_specific_weekday_of_month(1, 0, 9, year),
        "Veteranentag": date(year, 11, 11),
        "Erntedankfest": find_specific_weekday_of_month(4, 3, 11, year),
        "Thanksgiving-Freitag": find_specific_weekday_of_month(4, 3, 11, year) + timedelta(days=1),
        "Heiligabend": date(year, 12, 24),
        "Weihnachtstag": date(year, 12, 25)
    }

    # Anpassung für Feiertage, die auf das Wochenende fallen
    if holidays["Neujahrstag"].weekday() == 6:  # Sonntag
        holidays["Neujahrstag"] = date(year, 1, 2)
    if holidays["Unabhängigkeitstag"].weekday() in [5, 6]:  # Samstag oder Sonntag
        holidays["Unabhängigkeitstag"] = date(year, 7, 3) if holidays["Unabhängigkeitstag"].weekday() == 5 else date(year, 7, 5)
    if holidays["Weihnachtstag"].weekday() in [5, 6]:  # Samstag oder Sonntag
        holidays["Weihnachtstag"] = date(year, 12, 24) if holidays["Weihnachtstag"].weekday() == 5 else date(year, 12, 26)

    return holidays.values()

In [116]:
# Erstellen einer Liste aller Tage von 2017 bis 2020
start_date = date(2017, 1, 1)
end_date = date(2020, 12, 31)
delta = end_date - start_date

dates = [start_date + timedelta(days=i) for i in range(delta.days + 1)]

# Erstellen einer Liste aller Feiertage von 2017 bis 2020
holiday_dates = []
for year in range(2017, 2021):
    holiday_dates.extend(get_holidays(year))

# Erstellen des Dictionary
data = {
    "date": [],
    "is_holiday": []
}

for day in dates:
    data["date"].append(day)
    data["is_holiday"].append(day in holiday_dates)

holiday_data = pd.DataFrame(data)

# Datentypen ändern
holiday_data.date = pd.to_datetime(holiday_data.date)

holiday_data.head()

,date,is_holiday
0,2017-01-01,False
1,2017-01-02,True
2,2017-01-03,False
3,2017-01-04,False
4,2017-01-05,False


<a id="5"></a>
## 5. Umfragedaten vorbereiten
Ignorierte Daten u.a.:
* Demografische Daten über die Bewohner wie Ethie, Alter, Geschlecht, Bildungsabschluss, Einkommen, etc.
* Informationen aus Spalten, die nur als Freitext vorliegen und nicht direkt verwendbar sind wie Anzahl der Stunden im Home Office, Veränderungsarbeiten am Haus (wie z.B. Reparaturen, Austausch von Geräten, Neuinstallationen, Renovierungen)
* Details über Geräte wie Hersteller, typischer Stomverbrauch, etc.

In [117]:
def convert_square_feet_to_square_meters(feet):
    """
    Funktion zur Umrechnung von Square Feet zu Quadratmeter (m²).
    :param foot: Fläche in Square Feet.
    :return: Fläche in Quadratmeter (m²).
    """
    meters = round(feet * 0.092903, 2)
    return meters

In [118]:
survey_data = pd.read_csv('../input/2023-12-20/audits_surveys/survey_2013_all_participants.csv')

# Duplikate filtern
survey_data = survey_data.groupby('dataid').last().reset_index()

# Jahreszahlen einheitlich als Integer speichern
survey_data['construction_year'] = survey_data.year_house_constructed.replace('1930 or earlier', '1930').astype('Int64')

# Überführung der Werte aus Spalte primary_residence in Bool-Werte
# Leere Zeilen sorgen dafür, dass man nicht direkt in True/False konvertieren kann
survey_data['is_primary_residence'] = survey_data.primary_residence.map({'Yes': True, 'No': False}).astype('boolean')

# Berechnung der Anzahl der Bewohner über die Altersklassen
age_columns = ['residents_under_5', 'residents_6_to_12', 'residents_13_to_18', 'residents_19_to_24',
               'residents_25_to_34', 'residents_35_to_49', 'residents_50_to_64', 'residents_older_65']
survey_data['n_residents'] = survey_data[age_columns].sum(axis=1).astype('Int64')

# Datentypen ändern
survey_data['n_rooms'] = survey_data.house_num_rooms.astype('Int64')

# Umrechnung der Fläche von Square Feet zu Quadratmeter
survey_data['total_area'] = survey_data.house_square_feet.apply(convert_square_feet_to_square_meters)

# Auswahl der relevanten Spalten
relevant_columns = ['dataid', 'construction_year', 'is_primary_residence', 'n_residents', 'n_rooms', 'total_area']
survey_data = survey_data[relevant_columns]

survey_data.sample(5)

,dataid,construction_year,is_primary_residence,n_residents,n_rooms,total_area
150,4874,2010,True,2,7,195.10
240,8124,1955,True,5,7,292.64
204,6927,1994,True,1,6,185.81
35,1169,2008,True,0,6,132.85
16,469,1995,True,2,13,252.05


<a id="6"></a>
## 6. Datensatz auf ausgewählte Häuser beschränken

Auswahlkriterien:
1. Beschränkung auf die 10 Häuser mit den meisten gemessenen Datenpunkten
1. Teilnahme an der Umfrage aus 2013, da hier die meisten relevanten Informationen gesammelt wurden

In [119]:
# Zählen der Vorkommen von dataid in main_data
counts = main_data.dataid.value_counts()

# Filtern basierend auf den dataids in survey_data
filtered_counts = counts[counts.index.isin(survey_data.dataid)]

# Top 10 dataids mit den meisten Vorkommen
top_10_dataids = filtered_counts.nlargest(10)

top_10_dataids

dataid
6139    35036
4031    35036
9922    35036
9278    35035
661     35032
7800    35020
4767    34959
3456    34932
5746    34736
1642    34648
Name: count, dtype: int64

<a id="7"></a>
## 7. Daten zusammenführen
Anmerkung: Da zu den ausgewählten Häusern keine Daten zur Innentemperatur vorliegen, wird dieser Schritt übersprungen.

In [120]:
main_data_filtered = main_data[main_data.dataid.isin(top_10_dataids.index)]

# Unlogische Werte aussortieren (könnte z.B. auf Stromausfall oder fehlerhafte Messung hindeuten)
main_data_filtered = main_data_filtered[main_data_filtered.use != 0]

# Stündlichen Durchschnitt für Werte bilden
main_data_filtered.set_index('timestamp', inplace=True)
main_data_hourly = main_data_filtered.groupby(['dataid', pd.Grouper(freq='H')]).mean().reset_index()

# Umfragedaten joinen
j1 = pd.merge(main_data_hourly, survey_data, on='dataid', how='left')

# Wetterdaten joinen
j2 = pd.merge(j1, weather_data, on='timestamp', how='left')

# Feiertagsdaten joinen
j2['date'] = pd.to_datetime(j2.timestamp.dt.date)
df = pd.merge(j2, holiday_data, on='date', how='left')

df.drop(columns=['date'], inplace=True)

# Entfernen von leeren Spalten
df.dropna(axis=1, how='all', inplace=True)

df.sample(5)

,dataid,timestamp,air1,air2,airwindowunit1,bathroom1,bathroom2,bedroom1,bedroom2,bedroom3,...,use,construction_year,is_primary_residence,n_residents,n_rooms,total_area,outdoor_temp,app_outdoor_temp,humidity,is_holiday
25715,3456,2018-12-19 09:00:00,0.00000,NaN,NaN,0.024,NaN,0.009,NaN,NaN,...,0.28050,2008,True,2,10,159.79,13.41,13.41,0.98,False
70548,9278,2018-02-04 23:00:00,0.03525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.46775,2009,True,0,4,NaN,11.47,11.47,0.92,False
54202,6139,2018-03-25 10:00:00,0.00400,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,...,0.77725,1969,True,5,9,260.13,20.78,21.32,0.92,False
54926,6139,2018-04-24 14:00:00,0.00400,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,...,1.18400,1969,True,5,9,260.13,29.17,29.18,0.44,False
54261,6139,2018-03-27 21:00:00,0.67025,NaN,NaN,NaN,-0.0005,NaN,NaN,NaN,...,3.04675,1969,True,5,9,260.13,16.96,16.98,0.87,False


In [121]:
# Als csv-Datei speichern
df.to_csv('austin10_dataset.csv', index=False)